# https://blog.akquinet.de/2017/10/25/predicting-house-prices-on-kaggle-a-gentle-introduction-to-data-science-part-ii/

Some information like basement may be empty indicating there is no basement for the building

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from sklearn.neural_network import MLPClassifier

In [198]:
house_data = pd.read_csv("train.csv").drop("Id", axis=1)
test_data = pd.read_csv("test.csv")
house_data.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [223]:
from sklearn.decomposition import PCA

In [199]:
house_data.shape

(1460, 80)

In [200]:
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [201]:
cols_with_missing_train = [col for col in house_data.columns
                     if house_data[col].isnull().any()]
print(cols_with_missing_train)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [202]:
cols_with_missing_test = [col for col in test_data.columns
                     if test_data[col].isnull().any()]
print(cols_with_missing_test)

['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']


In [203]:
test_data[cols_with_missing].isnull().sum() / test_data.shape[0] * 100

MSZoning         0.274160
LotFrontage     15.558602
Alley           92.666210
Utilities        0.137080
Exterior1st      0.068540
Exterior2nd      0.068540
MasVnrType       1.096642
MasVnrArea       1.028101
BsmtQual         3.015764
BsmtCond         3.084304
BsmtExposure     3.015764
BsmtFinType1     2.878684
BsmtFinSF1       0.068540
BsmtFinType2     2.878684
BsmtFinSF2       0.068540
BsmtUnfSF        0.068540
TotalBsmtSF      0.068540
BsmtFullBath     0.137080
BsmtHalfBath     0.137080
KitchenQual      0.068540
Functional       0.137080
FireplaceQu     50.034270
GarageType       5.209047
GarageYrBlt      5.346127
GarageFinish     5.346127
GarageCars       0.068540
GarageArea       0.068540
GarageQual       5.346127
GarageCond       5.346127
PoolQC          99.794380
Fence           80.123372
MiscFeature     96.504455
SaleType         0.068540
dtype: float64

In [204]:
house_data[cols_with_missing].isnull().sum() / house_data.shape[0] * 100

MSZoning         0.000000
LotFrontage     17.739726
Alley           93.767123
Utilities        0.000000
Exterior1st      0.000000
Exterior2nd      0.000000
MasVnrType       0.547945
MasVnrArea       0.547945
BsmtQual         2.534247
BsmtCond         2.534247
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtFinSF1       0.000000
BsmtFinType2     2.602740
BsmtFinSF2       0.000000
BsmtUnfSF        0.000000
TotalBsmtSF      0.000000
BsmtFullBath     0.000000
BsmtHalfBath     0.000000
KitchenQual      0.000000
Functional       0.000000
FireplaceQu     47.260274
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageCars       0.000000
GarageArea       0.000000
GarageQual       5.547945
GarageCond       5.547945
PoolQC          99.520548
Fence           80.753425
MiscFeature     96.301370
SaleType         0.000000
dtype: float64

In [205]:
house_data = house_data.drop(cols_with_missing_test, axis = 1)
house_data

,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition,SalePrice
0,60,8450,Pave,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,61,0,0,0,0,0,2,2008,Normal,208500
1,20,9600,Pave,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,...,0,0,0,0,0,0,5,2007,Normal,181500
2,60,11250,Pave,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,...,42,0,0,0,0,0,9,2008,Normal,223500
3,70,9550,Pave,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,...,35,272,0,0,0,0,2,2006,Abnorml,140000
4,60,14260,Pave,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,...,84,0,0,0,0,0,12,2008,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,Pave,Reg,Lvl,Inside,Gtl,Gilbert,Norm,Norm,...,40,0,0,0,0,0,8,2007,Normal,175000
1456,20,13175,Pave,Reg,Lvl,Inside,Gtl,NWAmes,Norm,Norm,...,0,0,0,0,0,0,2,2010,Normal,210000
1457,70,9042,Pave,Reg,Lvl,Inside,Gtl,Crawfor,Norm,Norm,...,60,0,0,0,0,2500,5,2010,Normal,266500
1458,20,9717,Pave,Reg,Lvl,Inside,Gtl,NAmes,Norm,Norm,...,0,112,0,0,0,0,4,2010,Normal,142125


In [206]:
test_data = test_data.drop(cols_with_missing_test, axis = 1)
test_data

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1461,20,11622,Pave,Reg,Lvl,Inside,Gtl,NAmes,Feedr,...,140,0,0,0,120,0,0,6,2010,Normal
1,1462,20,14267,Pave,IR1,Lvl,Corner,Gtl,NAmes,Norm,...,393,36,0,0,0,0,12500,6,2010,Normal
2,1463,60,13830,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,212,34,0,0,0,0,0,3,2010,Normal
3,1464,60,9978,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,360,36,0,0,0,0,0,6,2010,Normal
4,1465,120,5005,Pave,IR1,HLS,Inside,Gtl,StoneBr,Norm,...,0,82,0,0,144,0,0,1,2010,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,1936,Pave,Reg,Lvl,Inside,Gtl,MeadowV,Norm,...,0,0,0,0,0,0,0,6,2006,Normal
1455,2916,160,1894,Pave,Reg,Lvl,Inside,Gtl,MeadowV,Norm,...,0,24,0,0,0,0,0,4,2006,Abnorml
1456,2917,20,20000,Pave,Reg,Lvl,Inside,Gtl,Mitchel,Norm,...,474,0,0,0,0,0,0,9,2006,Abnorml
1457,2918,85,10441,Pave,Reg,Lvl,Inside,Gtl,Mitchel,Norm,...,80,32,0,0,0,0,700,7,2006,Normal


Project Plans:
1. Use PCA to identify key features
2. Use ordinal encoding for categories and see the performance
3. Replace missing values with more accurate values instead of deleting them
4. Numerical features that are secretly categorical

In [207]:
string_data = house_data.select_dtypes(include=['object'])
le = preprocessing.LabelEncoder()
for column in string_data:
    le.fit(house_data[column])   
    house_data[column] = le.transform(house_data[column])
    test_data[column] = le.transform(test_data[column])

In [208]:
house_data['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [219]:
clf = MLPClassifier(solver='adam', early_stopping = True, hidden_layer_sizes=(46, 20), random_state=0, max_iter = 2000, batch_size=200, shuffle=True)
clf.fit(house_data.iloc[:, :46].values ,house_data['SalePrice'].values)

MLPClassifier(batch_size=200, early_stopping=True, hidden_layer_sizes=(46, 20),
              max_iter=2000, random_state=0)

In [220]:
pred = clf.predict(test_data[house_data.iloc[:, :46].columns].values)
pred

array([135000, 171000, 153575, ..., 311872, 135000, 230500])

In [221]:
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)

(array([ 84900,  92000, 110000, 124900, 129500, 132500, 134800, 135000,
        136905, 145250, 150900, 153575, 168500, 171000, 179900, 186000,
        207500, 221500, 230500, 234000, 255500, 311872, 345000]),
 array([ 34,  14,   1,  75, 121,   4,   5, 334,   6, 118,  14,  87,  27,
          1,  55,   3,  43,  50, 395,   2,   5,  61,   4]))

In [222]:
output.to_csv("out_2.csv", index=None)

In [280]:
pca = PCA()
pca.fit_transform(house_data)
pca.feature_names_in_

array(['MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
       '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold',
       'YrSold', 'SaleCondition', 'SalePrice'], dtype=object)

In [299]:
initial_feature_names = house_data.columns

In [300]:
most_important = [np.abs(pca.components_[i]).argmax() for i in range(36)]
most_important_names = [initial_feature_names[most_important[i]] for i in range(36)]

In [301]:
dic = {'PC{}'.format(i): most_important_names[i] for i in range(36)}

df = pd.DataFrame(dic.items())
df

,0,1
0,PC0,SalePrice
1,PC1,LotArea
2,PC2,2ndFlrSF
3,PC3,MiscVal
4,PC4,1stFlrSF
5,PC5,WoodDeckSF
6,PC6,OpenPorchSF
7,PC7,EnclosedPorch
8,PC8,LowQualFinSF
9,PC9,ScreenPorch


In [302]:
train_2 = house_data[df[1]]
train_2_final = train_2.drop('SalePrice', axis = 1)

In [365]:
clf = MLPClassifier(solver='adam', early_stopping = True, hidden_layer_sizes=(40, 10), random_state=1, max_iter = 2000, batch_size=500, shuffle=True)
clf.fit(train_2_final.values ,train_2['SalePrice'].values)

MLPClassifier(batch_size=500, early_stopping=True, hidden_layer_sizes=(40, 10),
              max_iter=2000, random_state=1)

In [366]:
test_data[train_2_final.columns]

,LotArea,2ndFlrSF,MiscVal,1stFlrSF,WoodDeckSF,OpenPorchSF,EnclosedPorch,LowQualFinSF,ScreenPorch,PoolArea,...,Condition1,RoofStyle,OverallQual,ExterCond,LandContour,BldgType,BldgType,Foundation,Fireplaces,BedroomAbvGr
0,11622,0,0,896,140,0,0,0,120,0,...,1,1,5,4,3,0,0,1,0,2
1,14267,0,12500,1329,393,36,0,0,0,0,...,2,3,6,4,3,0,0,1,0,3
2,13830,701,0,928,212,34,0,0,0,0,...,2,1,5,4,3,0,0,2,1,3
3,9978,678,0,926,360,36,0,0,0,0,...,2,1,6,4,3,0,0,2,1,3
4,5005,0,0,1280,0,82,0,0,144,0,...,2,1,8,4,1,4,4,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1936,546,0,546,0,0,0,0,0,0,...,2,1,4,4,3,3,3,1,0,3
1455,1894,546,0,546,0,24,0,0,0,0,...,2,1,4,4,3,4,4,1,0,3
1456,20000,0,0,1224,474,0,0,0,0,0,...,2,1,5,4,3,0,0,1,1,4
1457,10441,0,700,970,80,32,0,0,0,0,...,2,1,5,4,3,0,0,2,0,3


In [367]:
pred = clf.predict(test_data[train_2_final.columns].values)
pred

array([306000, 261500, 306000, ..., 116050, 200000, 200000])

In [368]:
output = pd.DataFrame(columns=['Id', 'SalePrice'])
output['Id'] = test_data['Id']
output['SalePrice'] = pred

np.unique(output['SalePrice'], return_index=False, return_inverse=False, return_counts=True, axis=None)

(array([ 78000, 105900, 108500, 116050, 125000, 125500, 132250, 139400,
        147500, 154500, 161000, 169500, 172400, 181000, 187500, 195400,
        200000, 206900, 214900, 215200, 220000, 239900, 248000, 251000,
        260000, 261500, 278000, 306000, 745000]),
 array([ 20,   1, 161,  14, 258,  71,  59,  15,  80,   7,   1,  52,  11,
         26,  14,   4, 227,   1,   9,   1,   3,   9, 157,  11,  25,   1,
         20, 105,  96]))

In [369]:
output.to_csv("out_8.csv", index=None)